In [55]:
import main as m

In [56]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import SGD

In [57]:
df = m.filter_data

In [58]:
# Split the data into input and target arrays for temperature prediction
X_temp = df[['Precipitation', 'Humidity', 'Pressure', 'Wind Speed']].values
y_temp = df['Temperature'].values

# Split the data into input and target arrays for precipitation prediction
X_prec = df[['Temperature', 'Humidity', 'Pressure', 'Wind Speed']].values
y_prec = df['Precipitation'].values


In [59]:
# We split the dataset into a training and test set.

#For Temperature:

x_temp_train , x_temp_test , y_temp_train , y_temp_test = train_test_split(X_temp, y_temp, test_size =0.2)

#For Precipitation

x_prec_train , x_prec_test , y_prec_train , y_prec_test = train_test_split(X_prec, y_prec, test_size =0.2)

In [60]:
# We reshape y_train and y_test since train_test_split
# outputs them as (nr_samples,) but we need
# (nr_samples ,1)

#for temperature
y_temp_train = y_temp_train.reshape((-1,1))
y_temp_test = y_temp_test.reshape((-1,1))

#for precipitation

y_prec_train = y_prec_train.reshape((-1,1))
y_prec_test = y_prec_test.reshape((-1,1))


In [61]:
#Now we reserve the test set and perform all
# preprocessing and training with the train set only. # The first preprocessing step is to stantardise our # features and target.
x_scaler = StandardScaler()
y_scaler = StandardScaler()

x_temp_train = x_scaler.fit_transform(x_temp_train)
x_prec_train = x_scaler.fit_transform(x_prec_train)

y_temp_train = y_scaler.fit_transform(y_temp_train)
y_prec_train = y_scaler.fit_transform(y_prec_train)

#test

x_temp_test = x_scaler.transform(x_temp_test)
x_prec_test = x_scaler.transform(x_prec_test)

y_temp_test = y_scaler.transform(y_temp_test)
y_prec_test = y_scaler.transform(y_prec_test)

In [63]:
#data transdormation for LSTM
#https://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/

x_temp_train = np.reshape(x_temp_train, (x_temp_train.shape[0], 1, x_temp_train.shape[1]))
x_temp_test = np.reshape(x_temp_test, (x_temp_test.shape[0], 1, x_temp_test.shape[1]))

x_prec_train = np.reshape(x_prec_train, (x_prec_train.shape[0], 1, x_prec_train.shape[1]))
x_prec_test = np.reshape(x_prec_test, (x_prec_test.shape[0], 1, x_prec_test.shape[1]))



In [65]:
x_temp_train.shape

(35060, 1, 4)

In [67]:
x_prec_test.shape

(8765, 1, 4)